In [1]:
import pandas as pd
import zipfile
import numpy as np
import matplotlib.pyplot as plt
import random
%matplotlib inline

In [2]:
# Unzip dataset
zip_ref = zipfile.ZipFile('political-social-media-posts.zip', 'r')
zip_ref.extractall('.')
zip_ref.close()

In [3]:
# Load in dataset with Latin-1 encoding
dataset = pd.read_csv('political_social_media.csv', encoding="ISO-8859-1")

In [ ]:
# shape
print(dataset.shape)
# head
print(dataset.head(20))

In [ ]:
# high-level descriptions of fields we're interested in
print(dataset.describe(include=[np.object])['message'])
print(dataset.describe(include=[np.object])['source'])
print(dataset.describe(include=[np.object])['bias'])

In [ ]:
fig1 = plt.figure(1)
fig1.set_size_inches(18.5, 10.5)
plt.subplot(221)
plt.title('Message Type')
pd.value_counts(dataset['message']).plot.bar()
plt.subplot(222)
plt.title('Source')
pd.value_counts(dataset['source']).plot.bar()
plt.subplot(223)
plt.title('Bias')
pd.value_counts(dataset['bias']).plot.bar()
plt.tight_layout()

In [ ]:
neutral_messages = dataset[(dataset.bias == 'neutral')]
partisan_messages = dataset[(dataset.bias == 'partisan')]

In [ ]:
fig2 = plt.figure(2)

fig2.set_size_inches(18.5, 10.5)
plt.subplot(221)
plt.title('Neutral Message Types')
neutral_counts = pd.value_counts(neutral_messages['message'])
neutral_counts.plot.bar()
plt.subplot(222)
plt.title('Partisan Message Types')
partisan_counts = pd.value_counts(partisan_messages['message'])
partisan_counts.plot.bar()
plt.tight_layout()
#########

fig3, axes = plt.subplots(nrows=1, ncols=2)
fig3.set_size_inches(18.5, 7.5)
neutral_df = neutral_counts.to_frame(name="messages")
normalized_neutral_df = (neutral_df-neutral_df.min())/(neutral_df.max()-neutral_df.min())
normalized_neutral_df.plot(kind='bar', ax=axes[0], sharex=False, sharey=False, title="Normalized Neutral Messages")
partisan_df = partisan_counts.to_frame(name="messages")
normalized_partisan_df = (partisan_df-partisan_df.min())/(partisan_df.max()-partisan_df.min())
normalized_partisan_df.plot(ax=axes[1], kind='bar',sharex=False, sharey=False, title="Normalized Partisan Messages")
plt.tight_layout()

In [ ]:
from keras.preprocessing.text import text_to_word_sequence
from keras.datasets import imdb

idx = imdb.get_word_index()
idx2word = {v: k for k, v in idx.items()}

def one_hot_encoding(message):
    return 0 if message == "attack" else 1

def word_to_idx(words):
    return [idx[word] if (word in idx and idx[word] <= 5000) else 5000 for word in words]

attack_or_support_df = dataset[(dataset.message == 'attack') | (dataset.message == 'support')][['message','text']]
attack_or_support_df = attack_or_support_df.reset_index(drop=True)
attack_or_support_df['message'] = attack_or_support_df['message'].apply(one_hot_encoding)
attack_or_support_df['text'] = attack_or_support_df['text'].apply(lambda x: word_to_idx(text_to_word_sequence(x)))
attack_or_support_df

In [ ]:
attack_or_support_df['text']

In [ ]:
lengths = [0]*len(attack_or_support_df['text'])
for index, x in enumerate(attack_or_support_df['text']):
    lengths[index] =len(x)
print("Mean %.2f words (%f)" % (np.mean(lengths), np.std(lengths)))

In [ ]:
attack_indices = attack_or_support_df.index[attack_or_support_df['message'] == 0].tolist()
support_indices = attack_or_support_df.index[attack_or_support_df['message'] == 1].tolist()
validation_indices = random.sample(range(len(attack_indices)), int(0.20*len(attack_indices))) + random.sample(range(len(support_indices)), int(0.20*len(support_indices)))


In [ ]:
len(validation_indices)

In [ ]:
y_test = attack_or_support_df.iloc[validation_indices]['message'].tolist()
x_test = attack_or_support_df.iloc[validation_indices]['text'].tolist()
y_train = attack_or_support_df[~attack_or_support_df.index.isin(validation_indices)]['message'].tolist()
x_train = attack_or_support_df[~attack_or_support_df.index.isin(validation_indices)]['text'].tolist()

In [ ]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import LSTM

In [ ]:
x_test = sequence.pad_sequences(x_test, maxlen=100)
x_train = sequence.pad_sequences(x_train, maxlen=100)

In [ ]:
seed = 7
np.random.seed(seed)

In [ ]:
mlp_model = Sequential()
mlp_model.add(Embedding(5001, 32, input_length=100))
mlp_model.add(Flatten())
mlp_model.add(Dense(250, activation='relu'))
mlp_model.add(Dense(1, activation='sigmoid'))
mlp_model.compile(loss='binary_crossentropy', optimizer='adam' , metrics=['accuracy'])
print(mlp_model.summary())
# Fit the model
mlp_model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=2, batch_size=128, verbose=2)
# Final evaluation of the model
mlp_scores = mlp_model.evaluate(x_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (mlp_scores[1]*100))

In [ ]:
conv_model = Sequential()
conv_model.add(Embedding(5001, 32, input_length=100))
conv_model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
conv_model.add(MaxPooling1D(pool_size=2))
conv_model.add(Flatten())
conv_model.add(Dense(250, activation='relu'))
conv_model.add(Dense(1, activation='sigmoid'))
conv_model.compile(loss='binary_crossentropy', optimizer='adam' , metrics=['accuracy'])
print(conv_model.summary())
conv_model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=2, batch_size=128, verbose=2)
conv_scores = conv_model.evaluate(x_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (conv_scores[1]*100))

In [ ]:
lstm_cnn_model = Sequential()
lstm_cnn_model.add(Embedding(5001, 32, input_length=100))
lstm_cnn_model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
lstm_cnn_model.add(MaxPooling1D(pool_size=2))
lstm_cnn_model.add(LSTM(100))
lstm_cnn_model.add(Dense(1, activation='sigmoid'))
lstm_cnn_model.compile(loss='binary_crossentropy' , optimizer='adam', metrics=['accuracy'])
print(lstm_cnn_model.summary())
lstm_cnn_model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=2, batch_size=128)
lstm_cnn_scores = lstm_cnn_model.evaluate(x_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (lstm_cnn_scores[1]*100))